# VLSP Qwen LoRA Notebook
# Fine-tuning Qwen với LoRA trên tập public_test.vi.txt (Medical Vietnamese)
# Tác giả: Hoàng Minh
# Mục tiêu: Tóm tắt văn bản y khoa tiếng Việt bằng Qwen + LoRA (PEFT)

## 0. Cài đặt các thư viện cần thiết (chạy 1 lần)

In [ ]:
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install transformers accelerate peft datasets sentencepiece sacrebleu python-pptx sentence-transformers nltk

## 1. Import các thư viện

In [ ]:
import os
import re
import json
import random
import math
import numpy as np
import pandas as pd
from pathlib import Path

from datasets import Dataset
import nltk
from nltk.tokenize import sent_tokenize

# Transformers & PEFT
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

# Evaluation
import sacrebleu
from sentence_transformers import SentenceTransformer, util

# PPTX
from pptx import Presentation
from pptx.util import Inches, Pt

## 2. Cấu hình đường dẫn

In [ ]:
DATA_PATH = Path('public_test.vi.txt')
OUTPUT_DIR = Path('qwen_lora_output')
OUTPUT_DIR.mkdir(exist_ok=True)

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

## 3. Đọc dữ liệu

In [ ]:
text = DATA_PATH.read_text(encoding='utf-8')
text = text.replace('\r\n', '\n')
records = [r.strip() for r in re.split(r"\n\s*\n", text) if r.strip()]
print(f"Số văn bản gốc: {len(records)}")
records[:3]

## 4. Xem thử vài mẫu

In [ ]:
for i, rec in enumerate(records[:5]):
    print(f"---- RECORD {i+1} ----")
    print(rec[:800])
    print("\n")

## 5. Tạo pseudo-target (dùng câu đầu làm tóm tắt ngắn)

In [ ]:
nltk.download('punkt', quiet=True)

examples = []
for rec in records:
    sents = sent_tokenize(rec, language='english')  # Vietnamese không có sẵn, dùng English tạm ổn
    if len(sents) >= 2:
        source = rec
        target = sents[0].strip()
        examples.append({'text': source, 'summary': target})

print(f"Số mẫu sau khi tạo pseudo-label: {len(examples)}")

## 6. Tạo HuggingFace Dataset + chia train/val

In [ ]:
ds = Dataset.from_list(examples)
train_test = ds.train_test_split(test_size=0.1, seed=SEED)
train_ds = train_test['train']
eval_ds = train_test['test']

print(f"Train: {len(train_ds)}, Eval: {len(eval_ds)}")

# Lưu preview
(OUTPUT_DIR / 'train_preview.json').write_text(json.dumps(train_ds[:5], ensure_ascii=False, indent=2), encoding='utf-8')

## 7. Load Qwen tokenizer & model (thay MODEL_NAME nếu cần)

In [ ]:
MODEL_NAME = os.getenv('QWEN_MODEL', 'Qwen/Qwen2.5-7B-Instruct')  # Thay bằng model bạn có quyền truy cập

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    print(f"Loaded tokenizer: {MODEL_NAME}")
except Exception as e:
    print("Không load được Qwen, fallback về gpt2 để test")
    tokenizer = AutoTokenizer.from_pretrained('gpt2')

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

## 8. Preprocessing

In [ ]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [f"<|im_start|>system\nBạn là một trợ lý y khoa tiếng Việt.<|im_end|>\n<|im_start|>user\nTóm tắt đoạn văn y khoa sau bằng tiếng Việt:\n{t}<|im_end|>\n<|im_start|>assistant\n" for t in examples['text']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['summary'], max_length=max_target_length, truncation=True, padding="max_length")
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_tokenized = train_ds.map(preprocess_function, batched=True, remove_columns=['text','summary'])
eval_tokenized = eval_ds.map(preprocess_function, batched=True, remove_columns=['text','summary'])

## 9. Load model + LoRA

In [ ]:
try:
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, trust_remote_code=True, device_map="auto")
    model.resize_token_embeddings(len(tokenizer))
except Exception as e:
    print("Không load được model lớn, dùng gpt2 để demo")
    model = AutoModelForCausalLM.from_pretrained('gpt2')
    model.resize_token_embeddings(len(tokenizer))

# LoRA config
peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"]
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

## 10. Trainer

In [ ]:
training_args = TrainingArguments(
    output_dir=str(OUTPUT_DIR / "checkpoint"),
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    logging_steps=50,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=False,
    bf16=False,
    push_to_hub=False,
    save_total_limit=2,
    report_to=[]  # tắt wandb nếu không cần
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=eval_tokenized,
    tokenizer=tokenizer,
)

## 11. Huấn luyện (bỏ comment để chạy)

In [ ]:
# trainer.train()
# trainer.save_model(OUTPUT_DIR / "final_model")

## 12. Tạo báo cáo và slide

In [ ]:
def write_report():
    md = f"""# VLSP Medical Domain — Fine-tuning Qwen với LoRA

## Dữ liệu
- Số mẫu train: {len(train_ds)}
- Số mẫu eval: {len(eval_ds)}

## Model
- Base model: {MODEL_NAME}
- LoRA: r=8, alpha=32

## Kết quả
- Chưa chạy đánh giá (cần chạy trainer.train() trước)
"""
    (OUTPUT_DIR / "report.md").write_text(md, encoding="utf-8")

def create_pptx():
    prs = Presentation()
    slide = prs.slides.add_slide(prs.slide_layouts[0])
    slide.shapes.title.text = "VLSP Medical — Fine-tuning Qwen với LoRA"
    slide.placeholders[1].text = "Hoàng Minh\nAuto-generated"

    s = prs.slides.add_slide(prs.slide_layouts[1])
    s.shapes.title.text = "Tổng quan"
    s.shapes.placeholders[1].text_frame.text = f"Train: {len(train_ds)} mẫu\nEval: {len(eval_ds)} mẫu\nModel: {MODEL_NAME}"

    prs.save(OUTPUT_DIR / "slides.pptx")

write_report()
create_pptx()

print("Đã tạo báo cáo và slide!")

## Hoàn tất!
Notebook đã sẵn sàng để chạy trên Google Colab, Kaggle hoặc máy có GPU.
Chúc bạn đạt điểm cao VLSP!